In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

In [ ]:
#loading images and annotations
X = []
xmins =[]
ymins = []
xmaxs = []
ymaxs = []
for i in range(895):
    impath = '/content/drive/My Drive/TomatoDetection/images/tomato' + str(i) + '.png'
    img = cv.imread(impath)
    X.append(img)
    
    etpath = '/content/drive/My Drive/TomatoDetection/annotations/tomato'+ str(i) +'.xml'
    tree = ET.parse(etpath)
    root = tree.getroot()
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for child in root:
        if (child.tag == 'object'):
            xmin.append(int(child.find('bndbox').find('xmin').text))
            ymin.append(int(child.find('bndbox').find('ymin').text))
            xmax.append(int(child.find('bndbox').find('xmax').text))
            ymax.append(int(child.find('bndbox').find('ymax').text))
    xmins.append(xmin)
    ymins.append(ymin)
    xmaxs.append(xmax)
    ymaxs.append(ymax)

In [ ]:
len(X)

895

In [ ]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
ss = cv.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
cnt = 0
x_final = list()
y_final = list()
for i in range(len(X)):
  if(i%100 == 0):
    print("iteration :", i)
  try:
    image = X[i]
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    
    counter = 0
    falsecounter = 0
    for j in range(len(xmins[i])):
      timage = image[ymins[i][j]:ymaxs[i][j],xmins[i][j]:xmaxs[i][j]]
      timage = cv.resize(timage, (224,224), interpolation = cv.INTER_AREA)
      cv.imwrite('/content/drive/My Drive/TomatoDetection/imcrop3/t'+str(cnt)+'.jpg', timage)
      cnt += 1
      x_final.append(timage)
      y_final.append(1)
    for e,result in enumerate(ssresults):
      if e < 2000 :    
        x,y,w,h = result
        ious = []
        for j in range(len(xmins[i])):
          iou = get_iou({"x1":xmins[i][j],"x2":xmaxs[i][j],"y1":ymins[i][j],"y2":ymaxs[i][j]},{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
          ious.append(iou)
        if max(ious) > 0.95 :
          timage = image[y:y+h,x:x+w]
          timage = cv.resize(timage, (224,224), interpolation = cv.INTER_AREA)
          cv.imwrite('/content/drive/My Drive/TomatoDetection/imcrop3/t'+str(cnt)+'.jpg', timage)
          cnt += 1
          x_final.append(timage)
          y_final.append(1)

        if counter < 4:
          if max(ious) > 0.85 and iou < 0.95:
            timage = image[y:y+h,x:x+w]
            timage = cv.resize(timage, (224,224), interpolation = cv.INTER_AREA)
            x_final.append(timage)
            cv.imwrite('/content/drive/My Drive/TomatoDetection/imcrop3/t'+str(cnt)+'.jpg', timage)
            cnt += 1
            y_final.append(1)
            counter += 1
            # print("i :",i ," counter : ",counter)

        if falsecounter < 8:
          if max(ious) < 0.4:
            timage = image[y:y+h,x:x+w]
            timage = cv.resize(timage, (224,224), interpolation = cv.INTER_AREA)
            x_final.append(timage)
            cv.imwrite('/content/drive/My Drive/TomatoDetection/imcrop3/t'+str(cnt)+'.jpg', timage)
            cnt += 1
            y_final.append(0)
            falsecounter += 1
        if counter == 5 and falsecounter == 10:
          break
      
                    
  except Exception as e:
    print(e)
    continue

iteration : 0
iteration : 100
iteration : 200
iteration : 300
iteration : 400
iteration : 500
iteration : 600
iteration : 700
iteration : 800


In [ ]:
len(x_final)

12854

In [ ]:
np.array(y_final).sum()/len(x_final)

0.4429749494320834

In [ ]:
y_d = {'lables': y_final}
import pandas as pd
df = pd.DataFrame(y_d)
df.to_csv('/content/drive/My Drive/TomatoDetection/labels3.csv')

In [ ]:
x_final[0].shape

(224, 224, 3)